# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [138]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

#Uri configuration
uri = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
# Get File
output_data_file = "output_data/cities.csv"

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [139]:
city_df = pd.read_csv(output_data_file, dtype="object", encoding="utf-8")
city_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Tuktoyaktuk,69.4541,-133.0374,-9.4,84,75,10.36,CA,1616109435
1,Andrews,32.3187,-102.5457,62.6,17,1,5.03,US,1616109436
2,Requena,39.4883,-1.1004,46.0,91,49,3.87,ES,1616109436
3,Puerto Colombia,10.9878,-74.9547,78.8,78,75,12.66,CO,1616109436
4,Ushuaia,-54.8,-68.3,46.4,71,40,11.5,AR,1616109423
5,Port Elizabeth,-33.918,25.5701,69.8,83,20,23.02,ZA,1616109437
6,Atuona,-9.8,-139.0333,80.8,75,0,10.83,PF,1616109437
7,Alquízar,22.8058,-82.5842,82.4,57,20,6.91,CU,1616109437
8,Mataura,-46.1927,168.8643,55.99,92,93,1.99,NZ,1616109438
9,The Valley,18.217,-63.0578,78.01,69,40,10.36,AI,1616109438


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [140]:
gmaps.configure(api_key=g_key)

In [141]:
locations = city_df[["Lat", "Lng"]].astype(float)
rating = city_df["Humidity"].astype(float)
intensity = city_df["Humidity"].max()

In [142]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=rating, dissipating=False, max_intensity=int(intensity),point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [143]:
#A max temperature lower than 85 degrees but higher than 70.
#Wind speed less than 10 mph.
#Cloudiness is less than 10%.


clean_city_df = city_df.loc[(city_df['Max Temp'].astype(float) > 70) & (city_df['Max Temp'].astype(float) < 85) & (city_df['Wind Speed'].astype(float) < 10) & (city_df['Cloudiness'].astype(float) < 10)]
clean_city_df.count()

City          30
Lat           30
Lng           30
Max Temp      30
Humidity      30
Cloudiness    30
Wind Speed    30
Country       30
Date          30
dtype: int64

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [144]:

hotel_df = clean_city_df[['City','Lat','Lng','Max Temp','Humidity','Cloudiness','Wind Speed','Country']]
hotel_df["Hotel Name"] = ''

for index, row in hotel_df.iterrows():
    lat = row['Lat']
    lng = row['Lng']
    rslt = requests.get(f"{uri}location={lat},{lng}&radius=5000&type=lodging&keyword=hotel&key={g_key}").json()
    try:
        hotel_df["Hotel Name"].loc[(hotel_df['Lat'] == lat) & (hotel_df['Lng'] == lng)] = rslt["results"][0]["name"]
    except:
        hotel_df["Hotel Name"].loc[(hotel_df['Lat'] == lat) & (hotel_df['Lng'] == lng)] = "No Hotel"
        pass

In [145]:
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Hotel Name
40,Port Blair,11.6667,92.75,77.76,67,0,2.44,IN,Hotel Ami Grace
42,Himatnagar,23.6,72.95,77.0,41,4,4.61,IN,Dowlat Villas Palace The Heritage
60,Buckeye,33.3703,-112.5838,82.99,17,5,1.01,US,Westward Motel
93,Port Alfred,-33.5906,26.891,70.72,89,3,7.58,ZA,MyPond Hotel
101,Saint George,37.1041,-113.5841,71.01,32,1,6.91,US,Tru By Hilton St. George
108,Salalah,17.0151,54.0924,75.2,43,0,4.61,OM,Salalah Gardens Hotel
119,Gwadar,25.1216,62.3254,73.06,69,0,4.18,PK,Zaver Pearl Continental Hotel
120,García,25.8167,-100.5833,77.0,14,1,8.05,MX,Antique Hotel Boutique Garcia
122,Sur,22.5667,59.5289,79.07,41,0,8.84,OM,Sur Bandar Ayjah Hotel Apartments صور بندر الع...
131,Marawi,7.9986,124.2928,75.99,82,5,5.01,PH,Marawi Resort Hotel


In [146]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]].astype(float)

In [147]:
# Add marker layer ontop of heat map
fig = gmaps.figure()
rating = hotel_df["Humidity"].astype(float)
intensity = hotel_df["Humidity"].max()

heat_layer = gmaps.heatmap_layer(locations, weights=rating, dissipating=False, max_intensity=int(intensity),point_radius=1)
fig.add_layer(heat_layer)
# Add layer
hotel_layer = gmaps.marker_layer(
    locations,
    info_box_content=hotel_info
)
fig.add_layer(hotel_layer)

# Display figure


In [148]:
fig

Figure(layout=FigureLayout(height='420px'))